In [ ]:
!pip install igraph

In [ ]:
!pip install python-igraph 

  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=eb703336db390034d4e5f2144eb61cc58f2656b47baa737696613bcebc93f134
  Stored in directory: /root/.cache/pip/wheels/15/86/ef/b8bcdfbcb1c489771ad256c7cd1eb4971cdb7f3f670938b798
Successfully built python-igraph


In [ ]:
from igraph import *
from math import *
import sys
import numpy as np
import os
import glob
import random
from random import shuffle
from random import seed
import matplotlib.pyplot as plt
import time
import datetime
import collections
import networkx as nx
# from networkx import *

In [ ]:
# Constants
k = 5 		# Top k recomendations for a target user (Used for testing model-accuracy metrics)

In [ ]:
# load edge-list from file
def get_edge_list(dataset_path):
	data_file = open(dataset_path)
	edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
	data_file.close()
	return edge_list

# Get the similarity product for a path
# (product of path-step similarities)
def get_sim_product(sim, shortest_path):
	prod = 1
	for i in range(len(shortest_path) - 1):
		prod *= sim[shortest_path[i]][shortest_path[i+1]]
	return round(prod,3)

# Filter out, Sort and Get top-K predictions
def get_top_k_recommendations(graph,sim,i,k):
	return  sorted(filter(lambda x: i!=x and graph[i,x] != 1,range(len(sim[i]))) , key=lambda x: sim[i][x],reverse=True)[0:k]

# Convert edge_list into a set of constituent edges
def get_vertices_set(edge_list):
	res = set()
	for x,y in edge_list:
		res.add(x)
		res.add(y)
	return res

# **split dataset**

In [ ]:
def split_data(edge_list):
  random.seed(350)
  edge_list = list(edge_list)
  indexes = range(len((edge_list)))
  test_indexes = set(random.sample(indexes, len(indexes)//2)) # removing 50% edges from test data
  train_indexes = set(indexes).difference(test_indexes)
  test_list = [edge_list[i] for i in (test_indexes)]
  train_list = [edge_list[i] for i in train_indexes]
  return train_list,test_list

# **find precision and recall**

In [ ]:
import igraph
# Calculates accuracy metrics (Precision & Recall),
def print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set):
	precision = recall = c = 0
	for i in test_vertices_set:
		if i in train_vertices_set:
			actual_friends_of_i = set(test_graph.neighbors(i))

			# Handles case where test-data < k
			if len(actual_friends_of_i) < k:
				k2 = len(actual_friends_of_i)
			else:
				k2 = k

			top_k = set(get_top_k_recommendations(train_graph,sim,i,k2))

			precision += len(top_k.intersection(actual_friends_of_i))/float(k2)
			recall += len(top_k.intersection(actual_friends_of_i))/float(len(actual_friends_of_i))
			c += 1
	print ("Precision is : " + str(precision/c))
	print ("Recall is : " + str(recall/c))

# **Similarity metrics**

In [ ]:
def similarity(graph, i, j, method):
	if method == "common_neighbors":
		return len(set(graph.neighbors(i)).intersection(set(graph.neighbors(j))))
	elif method == "jaccard":
		return len(set(graph.neighbors(i)).intersection(set(graph.neighbors(j))))/float(len(set(graph.neighbors(i)).union(set(graph.neighbors(j)))))
	elif method == "adamic_adar":
		return sum([1.0/math.log(graph.degree(v)) for v in set(graph.neighbors(i)).intersection(set(graph.neighbors(j)))])
	
	elif method == "shortest_path":
		graph2 = graph.to_networkx()	
		if(nx.has_path(graph2,i,j)):
			a = nx.algorithms.shortest_path_length(graph2,i,j)		
		else:
			a = 100000
			print(a)
		return a
		# return 0
		# igraph._igraph.GraphBase()
		# a = igraph.Graph.shortest_paths(i,j)
		# print(a)
		# return a

# **Link Prediction Methods**

In [ ]:
def prediction(edge_list,method):
	train_list, test_list = split_data(edge_list)
	train_graph = Graph(train_list)
	test_graph = Graph(test_list)
	train_n =  train_graph.vcount() # This is maximum of the vertex id + 1
	train_vertices_set = get_vertices_set(train_list) # Need this because we have to only consider target users who are present in this train_vertices_set
	test_vertices_set = get_vertices_set(test_list) # Set of target users

	sim = [[0 for i in range(train_n)] for j in range(train_n)]
	if method !="shortest_path":
		for i in range(train_n):
			for j in range(train_n):
				if i!=j and i in train_vertices_set and j in train_vertices_set:
					sim[i][j] = similarity(train_graph,i,j,method) 
	else:
		sp = {}
		for i in train_vertices_set:
			sp[i] = train_graph.get_shortest_paths(i)

		for i in range(train_n):
			for j in range(train_n):
				if i!=j and i in train_vertices_set and j in train_vertices_set:
					sim[i][j] = sp[i][j]
	# else:
	# 	graph2 = train_graph.to_networkx()
	# 	sp = dict(nx.shortest_path_length(graph2))
	# 	print(sp.get('1'))

	print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set)

In [ ]:
def main():
	# Command line argument parsing
	method = "common_neighbors"
	dataset_path = "/content/drive/MyDrive/Complex_Networks/HW02/facebook_combined.txt"
	edge_list = get_edge_list(dataset_path)
	if method == "common_neighbors" or method == "jaccard" or method == "adamic_adar"  or method =="shortest_path":
		prediction(edge_list,method)	
	else:
		print ("please enter a valid method")

In [ ]:
main()

Precision is : 0.566554170993422
Recall is : 0.2014233086674809
